<a href="https://colab.research.google.com/github/PanasRojpattanadecha/bsc_dpdm24/blob/main/Thal_Group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
df = pd.read_csv('/content/drive/MyDrive/dataset/train_set.csv')

In [76]:
df.head(10)

,id number,Xn,Hb(Gm%),Hct(%),MCV(fl),A2 (2-3%),HbF (2%),HbA (95-97%),RBC,MCH,MCHC,MCV-RBC-3*HGB,MCV-RBC-5*HGB,|80-MCV|,|27-MCH|,|80-MCV|*|27-MCH|,Thal_Group
0,67d169,25.0,11.1,34.0,78.0,3.2,0.0,55.9,4.358974,25.464706,32.647059,40.341026,18.141026,2.0,1.535294,3.070588,Abnormal_variants
1,67l168,32.0,13.9,42.0,86.0,3.7,0.3,54.8,5.197674,28.859060,33.557047,35.802326,5.802326,6.0,1.859060,11.154362,Abnormal_variants
2,67i267,43.0,10.1,31.0,78.0,2.6,0.3,56.2,3.974359,25.412903,32.580645,43.725641,23.525641,2.0,1.587097,3.174194,Abnormal_variants
3,67e248,33.0,12.9,38.0,76.0,3.0,0.2,61.2,5.000000,25.800000,33.947368,32.300000,6.500000,4.0,1.200000,4.800000,Abnormal_variants
4,67h277,23.0,12.9,40.0,78.0,3.1,0.0,55.6,5.128205,25.155000,32.250000,34.171795,8.371795,2.0,1.845000,3.690000,Abnormal_variants
5,67h283,57.0,9.8,32.0,76.0,3.7,0.0,64.4,4.210526,23.275000,30.625000,42.389474,22.789474,4.0,3.725000,14.900000,Abnormal_variants
6,67d341,74.0,10.1,31.0,78.0,2.3,0.0,76.2,3.974359,25.412903,32.580645,43.725641,23.525641,2.0,1.587097,3.174194,Abnormal_variants
7,67k76,39.0,10.1,31.0,78.0,3.7,0.0,52.3,3.974359,25.412903,32.580645,43.725641,23.525641,2.0,1.587097,3.174194,Abnormal_variants
8,66q131,78.0,8.9,27.0,84.0,3.9,0.0,54.5,3.214286,27.688889,32.962963,54.085714,36.285714,4.0,0.688889,2.755556,Abnormal_variants
9,66s257,NaN,11.6,31.7,82.0,3.7,0.0,52.2,3.865854,30.006309,36.231884,43.334146,20.134146,2.0,3.006309,6.012618,Abnormal_variants


In [50]:
!pip install imbalanced-learn


In [77]:
df.isnull().sum()

,0
id number,0
Xn,435
Hb(Gm%),7
Hct(%),7
MCV(fl),7
A2 (2-3%),133
HbF (2%),116
HbA (95-97%),116
RBC,7
MCH,7


In [99]:
df.dropna(inplace=True)

In [79]:
df.isnull().sum()

,0
id number,0
Xn,0
Hb(Gm%),0
Hct(%),0
MCV(fl),0
A2 (2-3%),0
HbF (2%),0
HbA (95-97%),0
RBC,0
MCH,0


In [80]:
#แปลงค่าหมวดหมู่ของ `Thal_Group` เป็นตัวเลข
label_encoder = LabelEncoder()
df["Thal_Group"] = label_encoder.fit_transform(df["Thal_Group"])

In [81]:
# เลือกฟีเจอร์ที่สำคัญ
features = df.columns.drop("Thal_Group")
X = df[features]
y = df["Thal_Group"]

In [82]:
# แบ่งข้อมูลเป็น Train/Test (75:25)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10, stratify=y)

In [83]:
print(X_train.dtypes)  # ดูประเภทข้อมูลของแต่ละคอลัมน์
print(X_train.select_dtypes(include=["object"]).head())  # ดูว่ามีคอลัมน์ข้อความหรือไม่


id number             object
Xn                   float64
Hb(Gm%)              float64
Hct(%)               float64
MCV(fl)              float64
A2 (2-3%)            float64
HbF (2%)             float64
HbA (95-97%)         float64
RBC                  float64
MCH                  float64
MCHC                 float64
MCV-RBC-3*HGB        float64
MCV-RBC-5*HGB        float64
|80-MCV|             float64
|27-MCH|             float64
|80-MCV|*|27-MCH|    float64
dtype: object
     id number
2906     66r57
4288    66z219
3833    66s123
2045    66r308
2304    66z239


In [84]:
X_train = X_train.drop(columns=["id number"])
X_test = X_test.drop(columns=["id number"])


In [85]:
print(X_train.dtypes)


Xn                   float64
Hb(Gm%)              float64
Hct(%)               float64
MCV(fl)              float64
A2 (2-3%)            float64
HbF (2%)             float64
HbA (95-97%)         float64
RBC                  float64
MCH                  float64
MCHC                 float64
MCV-RBC-3*HGB        float64
MCV-RBC-5*HGB        float64
|80-MCV|             float64
|27-MCH|             float64
|80-MCV|*|27-MCH|    float64
dtype: object


In [86]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [88]:
# เช็คจำนวนตัวอย่างในแต่ละคลาส
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

{0: 13, 1: 613, 2: 292, 3: 2, 4: 705, 5: 1221}


In [89]:
#ใช้ SMOTE เพื่อแก้ปัญหาคลาสไม่สมดุล (กำหนดค่าเฉพาะคลาสที่ต้องเพิ่ม)
smote = SMOTE(sampling_strategy={3: 10, 0: 50}, k_neighbors=1, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

In [90]:
#ตั้งค่า Random Forest และปรับแต่งพารามิเตอร์ด้วย RandomizedSearchCV
rf_model = RandomForestClassifier(class_weight="balanced", random_state=10)
param_dist = {
    "n_estimators": [100, 200, 300],
    "max_depth": [10, 15, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True]
}
random_search = RandomizedSearchCV(rf_model, param_dist, n_iter=10, cv=5, scoring="f1_weighted", random_state=10)
random_search.fit(X_train_resampled, y_train_resampled)

RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(class_weight='balanced',
                                                    random_state=10),
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [10, 15, None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=10, scoring='f1_weighted')

In [91]:
# ใช้โมเดลที่ดีที่สุด
best_rf_model = random_search.best_estimator_

In [125]:
# ทดสอบโมเดล
y_pred = best_rf_model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f" Accuracy บน Test Set: {accuracy:.4f}")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

 Accuracy บน Test Set: 0.9347
                    precision    recall  f1-score   support

 Abnormal_variants       1.00      1.00      1.00         4
Alpha_thal_related       0.83      0.90      0.86       204
 Beta_thal_related       0.99      0.98      0.98        98
 Complex_genotypes       0.00      0.00      0.00         1
       HbE_related       0.98      0.99      0.99       235
            Normal       0.95      0.91      0.93       407

          accuracy                           0.93       949
         macro avg       0.79      0.80      0.79       949
      weighted avg       0.94      0.93      0.93       949



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [93]:
# โหลดและเตรียมข้อมูล X_test.csv
test_file = "/content/drive/MyDrive/dataset/X_test.csv"
df_test = pd.read_csv(test_file)

In [98]:
df_test.isnull().sum()

,0
id number,0
Xn,187
Hb(Gm%),2
Hct(%),2
MCV(fl),1
A2 (2-3%),62
HbF (2%),59
HbA (95-97%),59
RBC,2
MCH,2


In [100]:
df_test.dropna(inplace=True)

In [101]:
df_test.isnull().sum()

,0
id number,0
Xn,0
Hb(Gm%),0
Hct(%),0
MCV(fl),0
A2 (2-3%),0
HbF (2%),0
HbA (95-97%),0
RBC,0
MCH,0


In [102]:
# เก็บ id number ไว้ใช้ภายหลัง
id_numbers = df_test["id number"]


In [103]:
# เลือกเฉพาะฟีเจอร์ที่ใช้
df_test = df_test[features]

In [124]:
# ตรวจสอบว่า "id number" อยู่ใน df_test หรือไม่
if "id number" in df_test.columns:
    id_numbers = df_test["id number"]
    df_test = df_test.drop(columns=["id number"])
else:
    # ถ้าไม่พบ id number ให้ใช้ index แทน
    id_numbers = df_test.index
    print(" ไม่พบ `id number` ใน X_test.csv, ใช้ index แทน")

# ตรวจสอบว่า df_test มีฟีเจอร์ที่ตรงกับ Training Set
missing_features = set(features) - set(df_test.columns) - {'id number'}
if missing_features:
    raise ValueError(f" ฟีเจอร์ที่ขาดหายใน X_test: {missing_features}")

extra_features = set(df_test.columns) - set(features)
if extra_features:
    print(f" พบฟีเจอร์เกินมาใน X_test: {extra_features} → จะถูกลบออก")
    df_test = df_test[features.drop('id number')]

 ไม่พบ `id number` ใน X_test.csv, ใช้ index แทน


In [114]:
# ตรวจสอบให้แน่ใจว่าข้อมูลเป็นตัวเลขทั้งหมด
df_test = df_test.astype(float)


In [122]:
# ปรับมาตรฐานข้อมูล (ใช้ StandardScaler จาก Training Data)
try:
    X_test_final_scaled = scaler.transform(df_test)
except ValueError as e:
    print(f" ข้อผิดพลาดในการปรับมาตรฐานข้อมูล: {e}")
    print("🔹 ตรวจสอบว่าฟีเจอร์ทั้งหมดเป็นตัวเลขและไม่มี NaN")
    print(df_test.info())
    raise

In [117]:
# ทำนายผลลัพธ์
y_test_pred = best_rf_model.predict(X_test_final_scaled)

In [118]:
# แปลงค่าตัวเลขกลับเป็นชื่อกลุ่ม `Thal_Group`
y_test_labels = label_encoder.inverse_transform(y_test_pred)

In [119]:
# สร้าง DataFrame สำหรับบันทึกผลลัพธ์
result_df = pd.DataFrame({"id number": id_numbers, "Thal_Group": y_test_labels})

In [121]:
# แสดงผลลัพธ์
print(result_df.head())

   id number Thal_Group
0          0     Normal
1          1     Normal
2          2     Normal
3          3     Normal
4          4     Normal


In [120]:
# บันทึกเป็น CSV
output_file = "/content/drive/MyDrive/dataset/y_pred_result.csv"
result_df.to_csv(output_file, index=False)
print(" ผลลัพธ์ถูกบันทึกเรียบร้อย: y_pred_result.csv")

 ผลลัพธ์ถูกบันทึกเรียบร้อย: y_pred_result.csv
